In [45]:
pip install selenium pandas pyarrow

   ---------------------------------------- 0.0/25.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/25.9 MB 825.8 kB/s eta 0:00:32
   ---------------------------------------- 0.2/25.9 MB 2.1 MB/s eta 0:00:13
    --------------------------------------- 0.4/25.9 MB 2.7 MB/s eta 0:00:10
    --------------------------------------- 0.6/25.9 MB 3.0 MB/s eta 0:00:09
   - -------------------------------------- 0.8/25.9 MB 3.0 MB/s eta 0:00:09
   - -------------------------------------- 1.0/25.9 MB 3.1 MB/s eta 0:00:09
   - -------------------------------------- 1.2/25.9 MB 3.4 MB/s eta 0:00:08
   -- ------------------------------------- 1.4/25.9 MB 3.5 MB/s eta 0:00:07
   -- ------------------------------------- 1.6/25.9 MB 3.6 MB/s eta 0:00:07
   -- ------------------------------------- 1.8/25.9 MB 3.6 MB/s eta 0:00:07
   -- ------------------------------------- 1.9/25.9 MB 3.5 MB/s eta 0:00:07
   --- ----


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

navegador = webdriver.Chrome()

navegador.get("https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br")

select_consulta = Select(navegador.find_element(By.XPATH,'//*[@id="segment"]'))
select_consulta.select_by_visible_text('Setor de Atuação')

# Aguardar até que o elemento da página esteja visível
elemento_pagina = WebDriverWait(navegador, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="selectPage"]'))
)

# Selecionar "120" no dropdown de página
select_pagina = Select(navegador.find_element(By.XPATH, '//*[@id="selectPage"]'))
select_pagina.select_by_visible_text('120')

In [47]:
# Encontrar a tabela no HTML
table = navegador.find_element(By.CLASS_NAME, 'table-responsive-md')

# Extrair os dados da tabela
data = []
headers = [header.text for header in table.find_elements(By.TAG_NAME, 'th')]

for row in table.find_elements(By.TAG_NAME, 'tr')[1:]:
    cells = row.find_elements(By.TAG_NAME, 'td')
    if len(cells) > 1:
        data.append([cell.text.strip() for cell in cells])

# Fechar o driver
navegador.quit()

# Verificar se o número de colunas é consistente
max_columns = len(headers)
for row in data:
    while len(row) < max_columns:
        row.append('')  # Adicionar células vazias conforme necessário

# Criar um DataFrame com os dados
df = pd.DataFrame(data, columns=headers)

# Salvar os dados em formato Parquet
df.to_parquet('dadosTabela.parquet', engine='pyarrow')

# Salvar os dados em formato CSV
df.to_csv('dadosTabela.csv', index=False)

print("Dados salvos com sucesso em dados.parquet e dados.csv")


PermissionError: [Errno 13] Permission denied: 'dadosTabela.csv'